In [35]:
%pip install pandas matplotlib seaborn scikit-learn -q



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt



df = pd.read_csv('./mcdonalds.csv')

In [37]:
df.head()

,_id,active,name,code,country,timezone,address,city,neighborhood,cep,...,services.mcDelivery,services.timeExtended,services.mcParty,services.playPlace,services.parking,services.wifi,services.wheelchairAccess,services.dessertCenter,services.shoppingCenter,generalHour.daysOfWeek
0,61d7fcbda283bebac83e89c9,True,SOBRADINHO - SBR,SBR,BR,America/Sao_Paulo,"QD 06 - Área Reservada 3, s/n",Brasília,Sobradinho,73070035,...,False,True,False,False,True,True,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '08:..."
1,62ed34bb7877dc2d6381ea1c,True,Drive Saída Norte - DSN,DSN,BR,America/Sao_Paulo,SPMN EPIA DF 03 Lote 02 Loja 01 Brasília - DF,Brasília,Lago Norte,71560100,...,True,True,False,False,False,False,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."
2,62ed31417877dcd78481e3a7,True,Shopping Iguatemi - BSI,BSI,BR,America/Sao_Paulo,"SHIN CA 4 Lote A, lojas 177/178 – Shopping Igu...",Brasília,Lago Norte,71503504,...,False,False,False,False,False,False,True,True,True,"[{'day': 'mon', 'timePeriods': [{'start': '11:..."
3,62ed30849b780128a34229f8,True,Shopping Boulevard - BSB,BSB,BR,America/Sao_Paulo,"STN Terminal Norte, S/N, Conjunto J, Loja T51 ...",Brasília,Asa Norte,70770100,...,True,False,False,False,False,False,True,False,True,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."
4,62ed2de97877dc4c1281d5ed,True,Asa Norte Eixinho - ASN,ASN,BR,America/Sao_Paulo,EQN 208/209 Bloco A – Lj. 50 – Asa Norte Bras...,Brasília,Asa Norte,70853400,...,True,True,False,False,False,False,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."


In [38]:
df.describe()

,distance,coordinates.longitude,coordinates.latitude,coordinates.lng,coordinates.lat
count,2343.000000,2343.000000,2343.000000,2343.000000,2343.000000
mean,1380.530839,-42.518490,-23.901153,-42.518490,-23.901153
std,1147.258006,9.610560,9.958499,9.610560,9.958499
min,34.005271,-63.876013,-56.114344,-63.876013,-56.114344
25%,898.639183,-47.068181,-23.694802,-47.068181,-23.694802
50%,920.855204,-46.546769,-23.480102,-46.546769,-23.480102
75%,1346.766388,-42.568615,-21.225657,-42.568615,-21.225657
max,11125.013013,46.701319,23.704880,46.701319,23.704880


In [39]:
df.shape

(2343, 39)

In [40]:
rename_dict = {
'id': 'Identificador',
'active': 'Ativo',
'name': 'Nome_do_lugar',
'code': 'Codigo',
'country': 'Pais',
'timezone': 'Fuso_horario',
'address': 'Endereco',
'city': 'Cidade',
'neighborhood': 'Bairro',
'services.mcDelivery': 'Delivery',
'services.timeExtended': 'Servicos_com_tempo_extimado',
'services.mcParty': 'Mcfestas_de_servicos',
'services.playPlace': 'Lugar_de_brincar',
'services.parking': 'Serviços_de_estacionamento',
'services.wifi': 'Wifi',
'services.wheelchairAccesse': 'Acessibilidade',
'services.dessertCenter': 'Centro_de_sobremesas',
'services.shoppingCenter': 'Shopping_center',
'generalHour.daysOfWeek': 'horas_dias_da_semana',
}
df = df.rename(columns=rename_dict)


In [41]:
df.head()

,_id,Ativo,Nome_do_lugar,Codigo,Pais,Fuso_horario,Endereco,Cidade,Bairro,cep,...,Delivery,Servicos_com_tempo_extimado,Mcfestas_de_servicos,Lugar_de_brincar,Serviços_de_estacionamento,Wifi,services.wheelchairAccess,Centro_de_sobremesas,Shopping_center,horas_dias_da_semana
0,61d7fcbda283bebac83e89c9,True,SOBRADINHO - SBR,SBR,BR,America/Sao_Paulo,"QD 06 - Área Reservada 3, s/n",Brasília,Sobradinho,73070035,...,False,True,False,False,True,True,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '08:..."
1,62ed34bb7877dc2d6381ea1c,True,Drive Saída Norte - DSN,DSN,BR,America/Sao_Paulo,SPMN EPIA DF 03 Lote 02 Loja 01 Brasília - DF,Brasília,Lago Norte,71560100,...,True,True,False,False,False,False,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."
2,62ed31417877dcd78481e3a7,True,Shopping Iguatemi - BSI,BSI,BR,America/Sao_Paulo,"SHIN CA 4 Lote A, lojas 177/178 – Shopping Igu...",Brasília,Lago Norte,71503504,...,False,False,False,False,False,False,True,True,True,"[{'day': 'mon', 'timePeriods': [{'start': '11:..."
3,62ed30849b780128a34229f8,True,Shopping Boulevard - BSB,BSB,BR,America/Sao_Paulo,"STN Terminal Norte, S/N, Conjunto J, Loja T51 ...",Brasília,Asa Norte,70770100,...,True,False,False,False,False,False,True,False,True,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."
4,62ed2de97877dc4c1281d5ed,True,Asa Norte Eixinho - ASN,ASN,BR,America/Sao_Paulo,EQN 208/209 Bloco A – Lj. 50 – Asa Norte Bras...,Brasília,Asa Norte,70853400,...,True,True,False,False,False,False,True,False,False,"[{'day': 'mon', 'timePeriods': [{'start': '10:..."


In [42]:
cols_to_drop = ['Centro_de_sobremesas', 'Shopping_center', 'Delivery']
df = df.drop(cols_to_drop, axis=1)


In [52]:
X = df[['Cidade', 'Bairro', 'Wifi']]
y = df['Nome_do_lugar']

X = df[['Cidade', 'Bairro', 'Wifi']]
y = df['Nome_do_lugar']

print(y.value_counts())
print(X['Cidade'].value_counts())
print(X['Bairro'].value_counts())
print(X['Wifi'].value_counts())





Nome_do_lugar
SOBRADINHO - SBR                    1
Drive Saída Norte - DSN             1
Shopping Iguatemi - BSI             1
Shopping Boulevard - BSB            1
Asa Norte Eixinho - ASN             1
                                   ..
SHOPPING ILHA SA CAVALCANTE Q101    1
SLU - BALCAO (COLD KIOSK)           1
SLU - CKR GOLDEN SHOPPING CALHAU    1
TestStore                           1
AUTODROMO INTERLAGOS - LOL          1
Name: count, Length: 2343, dtype: int64
Cidade
SAO PAULO         277
São Paulo         230
RIO DE JANEIRO    132
Rio de Janeiro     75
BELO HORIZONTE     45
                 ... 
DIVINOPOLIS         1
PIRASSUNUNGA        1
SERTAOZINHO         1
Campo Largo         1
Rio Branco          1
Name: count, Length: 423, dtype: int64
Bairro
Centro                159
CENTRO                 52
Barra da Tijuca        18
Bela Vista             12
Santo Amaro            12
                     ... 
MANAIRA                 1
Praia de Belas          1
Azenha                  1

In [44]:
X['Cidade'] = X['Cidade'].astype('category').cat.codes
X['Bairro'] = X['Bairro'].astype('category').cat.codes
X['Wifi'] = X['Wifi'].map({True: 1, False: 0})



C:\Users\jesie\AppData\Local\Temp\ipykernel_5100\1605179189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Cidade'] = X['Cidade'].astype('category').cat.codes
C:\Users\jesie\AppData\Local\Temp\ipykernel_5100\1605179189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Bairro'] = X['Bairro'].astype('category').cat.codes
C:\Users\jesie\AppData\Local\Temp\ipykernel_5100\1605179189.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)


c:\Users\jesie\Desktop\python\jesiel projetos\projeto_flask_colaborativo\projeto_flask_colaborativo\.venv\Lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  y_type = type_of_target(y, input_name="y")
c:\Users\jesie\Desktop\python\jesiel projetos\projeto_flask_colaborativo\projeto_flask_colaborativo\.venv\Lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  y_type = type_of_target(y, input_name="y")
c:\Users\jesie\Desktop\python\jesiel projetos\projeto_flask_colaborativo\projeto_flask_colaborativo\.venv\Lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  y_type = type_of_target(y, input_name="y")
c:\Users\jesie\Desktop\python\jesiel projetos\projeto_flask_colaborativo\projeto_flask_colaborativo\.venv\Lib\site-packages\sklear

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [47]:
y_pred = rf.predict(X_test)



In [49]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')

Acurácia do modelo: 0.00


c:\Users\jesie\Desktop\python\jesiel projetos\projeto_flask_colaborativo\projeto_flask_colaborativo\.venv\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\jesie\Desktop\python\jesiel projetos\projeto_flask_colaborativo\projeto_flask_colaborativo\.venv\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_pred = type_of_target(y_pred, input_name="y_pred")
